# Multidimensional arrays

IHE, Friday 20 Feb 2017

Theo Olsthoorn, 06-20440256, tolsthoorn@gmail.com

In [ ]:
import numpy as np
from scipy.special import exp1 as W # Theis well function
import matplotlib.pyplot as plt
%matplotlib notebook

What is it useful for?

Store multidimensional data, like

+ The head in a numerical network in 2 or 3D
+ time-distacne arrays, combining all times and distances, for instance of a drawdown
+ Multiple measurements

What we'll do

+ The differece between sequences (list, tuples) and numpy arrays (numerical arrays)
+ Show how to generate (with types), and manipulate multidimensional arrays and how to handle them
    .size,
    .shape,
    .reshape(),
    .hstack(),
    .vstack()
+ lists of list versus numerical arrays
+ slicing and indexing in arrays
    - slicing
    - logical indexing (show what's where)
+ Show broadcasting of arrays
+ The difference between an array and a matrix
+ Compute the drawdown versus time and show the drawns for many times and disctances in a single graph
    - a Theis well
    - a well and a mirror well
+ Compute the drawdown on a spatial grid
+ Contour the drawdown
    - line
    - full colors
    - colorbar( with title)
+ Compute the velocities on the grid
+ Show the velocities using quiver
+ Set up a system with a large number of linear equations and solve it.
    + regression
+ logical array indexing using polyline
    + polyline
    + np.spy


## Broadcasting

Broadcasting is automatic completion of differently shaped arrays to allow term by term computations.
This an extremly convinient and powerfull functionality of python's numpy module

In [ ]:
x = np.arange(11)
y = np.arange(11)
print(x)
print(y)

In [ ]:
x = x.reshape((1, 11))
x.T.shape
x.size
x.ndim

In [ ]:
y = y.reshape((11, 1))
print(x)
print(y)
print(x * y)
print(y * x)

In [ ]:
xy = x * y
print(xy)
print(xy.shape)
print(xy[2:3].shape)

## Compute function of t and r for all combinations simultaneously

Let's compute the drawdown due to a transient well for may times and distances simultaneously and show the drawdown for many times as s function of r and for many r as a function of time.

$$ s = \frac {Q_0} {4 \pi kD} W(u), \,\,\,\, u = \frac {r^2 S} {4 kD t} $$

In [ ]:
Q = 2400 # m3/d
S = 0.2
kD = 900 # m2/d

Nt = 30
Nr = 20
times = np.logspace(-3, 2, Nt)
distances = np.logspace(0, 3, Nr)

u = distances.reshape((1,Nr)) ** 2 * S / (4 * kD * times.reshape((Nt,1)))
s = Q / (4 * np.pi * kD) * W(u)

print(u.shape)

fig1, ax1 = plt.subplots()
ax1.set_xlabel('r [m]')
ax1.set_ylabel('s [m]')
ax1.set_xscale('log')
ax1.set_title('Drawdown vs r for many times')
for it, t in enumerate(times):
    ax1.plot(distances, s[it,:])
ax1.invert_yaxis()
plt.show()

fig2, ax2 = plt.subplots()
ax2.set_xlabel('t [d]')
ax2.set_ylabel('s [m]')
ax2.set_xscale('log')
ax2.set_title('Drawdown vs t for many distances r')
for ir, r in enumerate(distances):
    ax2.plot(times, s[:,ir])
ax2.invert_yaxis()
plt.show()


## Contouring your data

Compute the distance from an arbitrary point x, y to the well at x0, y0,
then extend x, y over a full grid and compute the drawdown in every point.
Finally contour the drawdowns.

We use the same well function and aquifer data as before.

In [ ]:
# aquifer
kD = 900 # m2/d
S = 0.15

# times
t = 1.2 # d

# coordinates
x = np.linspace(-500, 500, 101)
y = np.linspace(-500, 500, 101)
Nx = len(x)
Ny = len(y)
X = np.ones((len(y), 1)) * x[np.newaxis, :]
Y = y[:, np.newaxis] * np.ones((1, len(x)))

# wells
xw = [-100., 100.]
yw = [   0.,   0.]
rw = [ 0.25, 0.25]
Qw  = [1200., -1200.] # m3/d


s = np.zeros_like(X)
qr = np.zeros_like(X)
qx = np.zeros_like(X)
qy = np.zeros_like(X)

for x0, y0, r0, Q0 in zip(xw, yw, rw, Qw):  
    r = np.sqrt((X - x0)**2 + (Y - y0)**2)
    r[r==0] = r0
    u = r**2 * S / (4 * kD * t)
    s += Q0 / (4 * np.pi * kD) * W(u)
    q  = Q0 * np.exp(-u) / (2 * np.pi * r)
    qr += q 
    qx += q * (x0 - X)/r
    qy += q * (y0 - Y)/r

In [ ]:
xlim=(-300, 300)
ylim=(-300, 300)

fig, ax = plt.subplots()

ax.contour(x, y, s, np.linspace(np.min(s), np.max(s), 100))
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('well and mirror with contours and v-vectors')
ax.set_xlim(xlim)
ax.set_ylim(ylim)
plt.quiver(X, Y, qx, qy)
plt.show()

#### full colors

In [ ]:
xlim=(-150, 150)
ylim=(-150, 150)

fig, ax = plt.subplots()

cs = ax.contourf(x, y, s, np.linspace(np.min(s), np.max(s), 100))
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('well and mirror with contours and v-vectors')
ax.set_xlim(xlim)
ax.set_ylim(ylim)
plt.quiver(X, Y, qx, qy)
fig.colorbar(cs, ticks=np.linspace(-1.5, 1.5, 21))
plt.show()

### with plt.matshow()

In [ ]:
plt.matshow(s)
plt.show()

### Run a 3D finite difference groundwater model and show results

In [ ]:
mymodules = './modules'
import sys
if not mymodules in sys.path:
    sys.path.insert(1, mymodules)

import fdm
import mfgrid

In [ ]:
### Logical array indexing, data within a polyline (matplotlib path)
xGr = np.linspace(-500, 500, 101)
yGr = np.linspace(-500, 500, 101)
zGr = np.array([0, -10, -100])

gr = mfgrid.Grid(xGr, yGr, zGr)

fig, ax = plt.subplots()

#gr.plot_grid()

rch = 0.001 # m/d

k = gr.const(10.)
FQ = gr.const(0);  FQ[:,:,0] = rch * gr.Area
FH = gr.const(0.)
IBOUND = gr.const(1)
IBOUND[[0,-1], :, :] = -1
IBOUND[:, [0,-1], :] = -1

Out= fdm.fdm3(gr, (k, k, k), FQ, FH, IBOUND)

ax.contour(gr.xm, gr.ym, Out.Phi[:, :, 0], level=30)
plt.show()


In [ ]:
wells = np.array([[100, 175, -1200.],
                  [-205, 33, -800.],
                  [-80, 165, -600.]])

ixw = gr.ix(wells[:,0])
iyw = gr.iy(wells[:,1])
Qw  = wells[:,-1]

for ix, iy, Q in zip(ixw, iyw, Qw):
    FQ[iy, ix, -1] = Q 

In [ ]:
fig, ax = plt.subplots()

Out= fdm.fdm3(gr, (k, k, k), FQ, FH, IBOUND)

ax.contour(gr.xm, gr.ym, Out.Phi[:, :, 0], 30)
plt.show()


In [ ]:
import coords
crds = coords.coords * 2
crds[:,0] -=600
crds[:,1] -= 400
crds = np.vstack((crds, crds[0]))


fig, ax = plt.subplots()
ax.contour(gr.xm, gr.ym, Out.Phi[:, :, 0], 30)
ax.plot(crds[:,0], crds[:,1], lw=3)
plt.show()




In [ ]:
island = gr.inpoly(crds)

In [ ]:
fig, ax = plt.subplots()
ax.spy(island)

In [ ]:
ibnd = IBOUND[:,:, 0]
ibnd[~island] = -1
IBOUND[:,:,0] = ibnd
IBOUND[:,:,1] = ibnd

In [ ]:
Out= fdm.fdm3(gr, (k, k, k), FQ, FH, IBOUND)

fig, ax = plt.subplots()

ax.contour(gr.xm, gr.ym, Out.Phi[:, :, 0], 30)
ax.plot(crds[:,0], crds[:,1], 'b', lw=3)
plt.show()


In [ ]:
print('total across boundaries: ', np.sum(Out.Q[IBOUND==-1]))
print('total recharge:          ', np.sum(Out.Q[np.logical_and(FQ<0, IBOUND>0)]))
print('total wells:             ', np.sum(Out.Q[IBOUND== 1]))